# Automated Trading Algorithms


In [16]:
# Import libraries
from trading_session import * 
from financial_series.forex_data import * 


# Set GoogleChrome Driver
driv_r = '/home/rhdzmota/Downloads/chromedriver'

## Initialize web service

In [8]:

# create instance of trading_session object
ts = TradingSession(user='sd8892',pw='Mexico0r',driv_r=driv_r,alias='Trading Session')


## Tests 

In [10]:
ts.performOperation('compra','MEXCHEM','60','100')

In [11]:
ts.getExchangeRates()

{'Buy': 20.51, 'Sell': 20.52}

## Close 

In [4]:
ts.close()